# A Bank Call Center chatbot
Design a chatbot can guide and process the user's money transfer requests, based on different input from the user:

- I would like to transfer money.

- Can you transfer $500 from my savings to my checking?

- Actually could you change the amount to $250.

- Move it from my checking to my savings.

- Ok. let's forget about sending the money.

Input, user's message. Output: Guide the user until the transfer is made.


Design the system based on performance, ease of implemenation, maintainability and cost. 

The required_information to initiate the transfer: 
- from_account: 12345
- to_account: 56789
- amount: $250
- user_confirmation: yes

Process Design

<img src="flow.png" width=600px alt="Alternative text" />

In [10]:
import openai
from getpass import getpass

OPENAI_API_KEY = getpass("Enter your OpenAI API key: ")
MODEL = "gpt-4"
openai.api_key = OPENAI_API_KEY

In [11]:
import ast

def chat(messages: list, prompt: str) -> str:
    messages.append({"role": "user", "content": prompt})
    responses = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
    )
    return responses.choices[0]["message"]["content"]


def extractor(user_message: str) -> dict:
    messages = []
    system_message = """
    Role: Your role is to extract source account number, destination account number and transfer amount from the customer's message.
    """
    messages.append({"role": "system", "content": system_message})

    prompt = f"""
    You are an assitant to extract information from user's message. Extract the source account number, destination account number and transfer amount from the customer's message, output as a dictionary. 
    
    Example:
    User: "I want to transfer from account 123456."
    Assistant: {{"from_account": "123456", "to_account": "", "amount": ""}}
    User: "I want to transfer to account 654321."
    Assistant: {{"from_account": "", "to_account": "654321", "amount": ""}}
    User: "I want to transfer 100 dollars."
    Assistant: {{"from_account": "", "to_account": "", "amount": "100"}}
    User: {user_message}
    Assistant:
    """
    response = chat(messages, prompt)
    return ast.literal_eval(response)

def follow_up_asker(received_information: dict) -> str:
    messages = []
    system_message = """
     Role: You are bank call center assitant. Your goal is to collect the "from_account": "", "to_account": "", "amount": "" from the user.
    """
    messages.append({"role": "system", "content": system_message})

    prompt = f"""
     Role: You are bank call center assitant. Your goal is to collect the "from_account": "", "to_account": "", "amount": "" from the user.

    Here are some example: 
    User: "from_account": "", "to_account": "", "amount": ""
    Assistant: "Please tell me the source account, destination account and the amount you wish to transfer."
    User: "from_account": "1234", "to_account": "", "amount": ""
    Assistant: "What is the destination account number and the amount you wish to transfer?"
    User: "from_account": "1234", "to_account": "6543", "amount": ""
    Assistant: "What is the amount that you wish to transfer?"
    User: "from_account": "1234", "to_account": "6543", "amount": "100"
    Assistant: "Ok. I will help you with that."

    Let's start with the following conversation:
    User: "from_account": {received_information["from_account"]}, "to_account": {received_information["to_account"]}, "amount": {received_information["amount"]}.
    Assistant:
    """
    return chat(messages, prompt)

def confirmation_checker(user_message: str) -> str:
    messages = []
    system_message = """
     Role: You are bank call center assitant. Your goal is to confirm the transfer with the user.
    """
    messages.append({"role": "system", "content": system_message})

    prompt = f"""
     Role: You are bank call center assitant. Your goal is to confirm the transfer with the user. If the user replies positively, returns 1, otherwises returns 0.

    Here are some example:
    User: OK.
    Assistant: 1
    User: yes, please.
    Assistant: 1
    User: Wait a moment.
    Assistant: 0
    User: I don't want to transfer.
    Assistant: 0
    User: {user_message}
    Assistant:
    """
    return chat(messages, prompt)



### Chatbot Implementation

In [12]:
def collect_and_transfer() -> None:
    received = {'from_account': '', 'to_account': '', 'amount': ''}
    while not all(value for value in received.values()):
        user_message = input("User: ")
        print("User:", user_message)
        new_info_dict = extractor(user_message)
        for key, value in new_info_dict.items():
            if value:
                received[key] = value
        print("Received information:", received)
        if not all(value for value in received.values()):
            assitant_response = follow_up_asker(received)
            print("Assistant: ", assitant_response)
            print('\n\n')
        
    print("Assistant: ", f"""Okay, it looks like you'd like to transfer ${received["amount"]} from account number {received["from_account"]} to account number {received["to_account"]}. Is that correct?")
    """)

    user_message = input("User: ")
    print("User:", user_message)
    response = confirmation_checker(user_message)
    print("LLM Output:", response)
    if response == 1:
        print("Assistant: ", "Okay, I will help you with that.")
        # Execute the transfer    

In [20]:
collect_and_transfer()

User: hi
Received information: {'from_account': '', 'to_account': '', 'amount': ''}
Assistant:  "Please tell me the source account, destination account and the amount you wish to transfer."



User: 100
Received information: {'from_account': '', 'to_account': '', 'amount': '100'}
Assistant:  "Can you please tell me the source account and the destination account for the transfer?"



User: from 1234
Received information: {'from_account': '1234', 'to_account': '', 'amount': '100'}
Assistant:  "Thank you for providing your account number. However, I still need the destination account number. Could you please provide the account number you wish to transfer the amount to?"



User: to 4321
Received information: {'from_account': '1234', 'to_account': '4321', 'amount': '100'}
Assistant:  Okay, it looks like you'd like to transfer $100 from account number 1234 to account number 4321. Is that correct?")
    
User: no
LLM Output: 0


### Option2: Use OpenAI function call

The underlying llm will keep asking questions until all the **required** arguments are collected.

In [21]:
import openai
import json

def collect_transfer_openai_function():
  messages = []
  system_message = """
      Role: You are bank call center assitant. Your goal is to collect all the necessary information from the user and execute money transfer between bank accounts.
  """
  messages.append({"role": "system", "content": system_message})

  while True:
    user_message = input("User: ")
    print("User:", user_message)

    chat_history = ""
    for m in messages:
        chat_history += f"{m['role']}: {m['content']}\n"

    prompt = f"""
    Chat History:
    {chat_history}


    User: {user_message}
    Assistant:
    """

    get_transfer_description = {
    "name": "transfer_between_accounts",
      "description": "Transfer money between bank accounts",
      "parameters": {
        "type": "object",
        "properties": {
          "from": {
            "type": "string",
            "description": "The source account to transfer from."
          },
        "to": {
            "type": "string",
            "description": "The destination account to transfer to."
          },
        "amount": {
            "type": "string",
            "description": "The amount of money to transfer."
          }
        },
        "required": ["from", "to", "amount"]
      }
    }

    messages.append({"role": "user", "content": user_message})
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        functions = [get_transfer_description]
      )
    
    assistant_message = response["choices"][0]["message"]["content"]
    print("Assistant:", assistant_message)
    print("\n\n")

    if "function_call" in response["choices"][0]["message"]:
      arguments = response["choices"][0]["message"]["function_call"]["arguments"]
      print("Arguments:", json.loads(arguments))
      break


In [24]:
collect_transfer_openai_function()

User: 100
Assistant Message: I'm sorry for any confusion, but I need more context to assist you better. Are you trying to transfer $100 from one account to another? Could you please provide me with the details of the source and destination accounts?



User: 
Assistant Message: I'm sorry, but I didn't understand your request. Are you trying to transfer $100? If so, could you please provide me with the account numbers for both the source and destination accounts?



User: source account is 1234
Assistant Message: I understand you want to transfer $100 from account number 1234. Could you please provide me with the destination account number for the transfer?



User: actually from 0000
Assistant Message: Sure. We will now transfer the amount from the account: 0000. May I know the destination account?



User: 
Assistant Message: Sure, I understand you want to transfer $100 from account 0000. Could you please provide the destination account number?



User: destination is 432
Assistant Me